In [84]:
import numpy as np
import IPython

def KS(x, N, alpha=0.99, ref_length: int = 50):
    """ Karplus-Strong Algorithm is a simple digital feedback loop with an 
    internal buffer of  MM  samples. The buffer is filled with a set of initial values 
    and the loop, when running, produces an arbitraryly long output signal

    :param x: array of ints
    :param N: length of produced samples
    :param ref_length: decay buffer length
    """
    M = len(x)
    a = alpha ** (float(M) / ref_length)
    y = np.zeros(N)

    for n in np.arange(0, N):
        y[n] = (x[n] if n < M else 0) + a * (y[n - M] if n - M > 0 else 0)

    return y

def pitch(freq: float, duration: float, signal: np.array, sample_freq: int, alpha=0.99, ref_length=50):
    """ given a signal, generate a pitch using the signal
    with a frequency (Hz) and duration (Seconds)

    If frequency (f), and sample frequency (f_s) are given, the length of signal is

    desire_signal_length = f_s / f

    Hence if signal length cannot fulfill above, we should either cut or pad the signal.
    """
    total_sample_number = int(sample_freq * duration)
    desire_signal_length = int(sample_freq / freq)
    # pad or cur signal
    if len(signal) >= desire_signal_length:
        input_signal = signal[: desire_signal_length]
    else:  # pad
        input_signal = np.pad(signal, (0, desire_signal_length-len(signal)), 'constant')

    result = KS(input_signal, N=total_sample_number, alpha=alpha, ref_length=ref_length)

    return result


def note_freq(note: str):
    """ Given a note string, return the frequency
    For example:
    D2 -> 73.41619 Hz

    @param note:
    @return:
    """
    # general purpose function to convert a note  in standard notation
    #  to corresponding frequency
    if len(note) < 2 or len(note) > 3 or \
            note[0] < 'A' or note[0] > 'G':
        return 0
    if len(note) == 3:
        if note[1] == 'b':
            acc = -1
        elif note[1] == '#':
            acc = 1
        else:
            return 0
        octave = int(note[2])
    else:
        acc = 0
        octave = int(note[1])
    SEMITONES = {'A': 0, 'B': 2, 'C': -9, 'D': -7, 'E': -5, 'F': -4, 'G': -2}
    n = 12 * (octave - 4) + SEMITONES[note[0]] + acc
    f = 440 * (2 ** (float(n) / 12.0))
    return f

In [80]:
f      = 880
f_s    = 44000
dura   = 2
alpha  = 1

In [81]:
signal = np.random.rand(50)
output = pitch(f, dura, signal, f_s, alpha)
IPython.display.Audio(output, rate=f_s)

In [82]:
signal = np.ones(10)
output = pitch(f, dura, signal, f_s, alpha)
IPython.display.Audio(output, rate=f_s)

In [138]:
chord = {
    'C5': 4.0,
    'E5': 4.0,
    'G5': 4.0
}
color = np.random.rand(20)
duration = 2
f_s = 44000
alpha = 0.996
pitches = []
chord_signal =  np.zeros(duration*f_s)
for note, gain in chord.items():
    print(note, note_freq(note), gain)
    freq = note_freq(note)
    tmp = pitch(freq, duration, color, f_s, alpha)
    pitches.append(tmp)
    chord_signal += tmp*gain

C5 523.2511306011972 4.0
E5 659.2551138257398 4.0
G5 783.9908719634985 4.0


In [139]:
IPython.display.Audio(chord_signal, rate=f_s)

In [140]:
c, e, g = pitches

In [141]:
chord_signal.shape

(88000,)

In [142]:
IPython.display.Audio(c, rate=f_s)

In [143]:
IPython.display.Audio(e, rate=f_s)

In [144]:
IPython.display.Audio(g, rate=f_s)